In [1]:
import pandas as pd
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import imgaug as ia
import os
import json

# agumentation Module
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage


In [7]:
boilerplate = '''
{
    "info": {
        "year": "2022",
        "version": "3",
        "description": "Exported from roboflow.ai",
        "contributor": "",
        "url": "https://public.roboflow.ai/object-detection/undefined",
        "date_created": "2022-07-02T07:33:39+00:00"
    },
    "licenses": [
        {
            "id": 1,
            "url": "https://creativecommons.org/licenses/by/4.0/",
            "name": "CC BY 4"
        }
    ],
    "categories": [
        {
            "id": 0,
            "name": "Clean",
            "supercategory": "grains"
        },
        {
            "id": 1,
            "name": "Broken",
            "supercategory": "grains"
        },
        {
            "id": 2,
            "name": "Chalky",
            "supercategory": "grains"
        },
        {
            "id": 3,
            "name": "Discolored",
            "supercategory": "grains"
        },
        {
            "id": 4,
            "name": "Immature",
            "supercategory": "grains"
        }
    ],
    "images": [
    ],
    
    "annotations": [
       
    ]
    
}
'''

json_f = json.loads(boilerplate)
global category_id

In [8]:
''' Making Image Class '''
class Image_class:
    def __init__(self, path, id, width, height):
        self.path = path
        self.id = id
        self.width = width
        self.height = height
        self.annotations = 0

    def add_annotation(self, anno_id, bbox, area, seg = []):
        if self.path.split('\\')[-4] == 'Clean':
            category_id = 0
        elif self.path.split('\\')[-4] == 'Broken':
            category_id = 1
        elif self.path.split('\\')[-4] == 'Chalky':
            category_id = 2
        elif self.path.split('\\')[-4] == 'Discolored':
            category_id = 3
        elif self.path.split('\\')[-4] == 'Immature':
            category_id = 4


        annotation = {
            "id": int(anno_id),
            "image_id": int(self.id),
            "bbox": bbox,
            "categories": int(category_id),
            "area": int(area),
            "segmentation": seg
        }
        json_f['annotations'].append(annotation)

    def add_category(self, category):
        self.categories.append(category)

    def add_license(self, license):
        self.licenses.append(license)

    def to_json(self):
        image = {
            "id": int(self.id),
            "width": int(self.width),
            "height": int(self.height),
            "file_name": self.path.split('\\')[-1],
            "license": 1, 
            "Date_created": self.path.split('\\')[-2]
        }
        json_f['images'].append(image)
        
  

In [4]:
''' Function for dataframe vectorization '''

def change_rect(rect):
    x, y, w, h = rect
    return [int(x), int(y), int(x+w), int(y+h)]
    
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
def bounrects(rects):
    return [BoundingBox(x1=rect[0], y1=rect[1], x2=rect[2], y2=rect[3]) for rect in rects]
    


In [9]:
'''Makeing Agumentation Pipeline'''

import imgaug.augmenters as iaa

BoundingBox = []

# Sometimes(0.5, ...) applies the given augmenter in 50% of all cases,
sometimes = lambda aug: iaa.Sometimes(0.5, aug)

seq = iaa.Sequential([
        iaa.Fliplr(0.5), # horizontally flip 50% of all images
        # iaa.Flipud(0.2), # vertically flip 20% of all images
        # iaa.Multiply((1.2, 1.5)), # change brightness, doesn't affect BBs
        # sometimes(iaa.Crop(percent=(0, 0.1))),# crop some of the images by 0-10% of their height/width
        iaa.Sharpen(alpha=(0, 0.9), lightness=(0.85, 1.25)),


        # For brightness
        iaa.Multiply((0.85, 1.15)),

        iaa.LinearContrast((1, 1.1), per_channel=0.5),

        iaa.Affine(
            translate_px={"x": 40, "y": 60},
            scale=(0.8, 0.95),
            rotate=(-5, 5),
            shear=(-1, 1),
    
        ) # translate by 40/60px on x/y axis, and scale to 50-70%, affects BBs
    ])

In [10]:
import glob
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage

img_id = 0
anno_id = 0
for img_path in glob.glob(r"D:\datasets\image_agumentation\\*\*\*\*e.png") or glob.glob(r"D:\datasets\image_agumentation\\*\*\*\*a.png"):
    class_name = img_path.split('\\')[-4]
    date = img_path.split('\\')[-2]
    image = cv.imread(img_path)

    print(img_path)

    # print(json_f)
    df = pd.read_pickle(img_path[0:-17]+'\output.pkl')
    df['rect'] = df.straight_rect.apply(change_rect)

    varss = df.rect.tolist()
    print(len(varss))

    thei =[BoundingBox(x1=recto[0], y1=recto[1], x2=recto[2], y2=recto[3]) for recto in varss]
    bbs = BoundingBoxesOnImage([BoundingBox(x1=recto[0], y1=recto[1], x2=recto[2], y2=recto[3]) for recto in df.rect], shape=image.shape)
    
    og_image =Image_class(img_path, img_id, image.shape[1], image.shape[0])
    
    for index_no in range(len(df)):
        og_image.add_annotation(anno_id, varss[index_no], 0)
        anno_id += 1

    
    og_image.to_json()

    img_id += 1
    
    BouBox = []
    for rep_no in range (15):
        # Augment BBs and images.
        image_aug, bbs_aug = seq(image=image, bounding_boxes=bbs)
        
        name = img_path[0:-4]+  str(rep_no) +'.png'

        # cv.imshow("image3", image_before)
        cv.imwrite(name, image_aug)

        # Makeing its its annotation
        derva = Image_class(name, img_id, image_aug.shape[1], image_aug.shape[0])


        for i in range(len(bbs.bounding_boxes)):
            after = bbs_aug.bounding_boxes[i]
            
            og_image.add_annotation(anno_id, [int(after.x1), int(after.y1), int(after.x2-after.x1), int(after.y2-after.y1)], int((after.x2-after.x1)*(after.y2-after.y1)))
            anno_id += 1
            
        derva.to_json()
        img_id += 1
        

        
    
    with open(img_path[0:-17]+'annotation.json', 'w') as file:
        json.dump(json_f,file ,indent=2)

    
    



D:\datasets\image_agumentation\Broken\Scan 1\14-02-2022_11_03_55\scanned_image.png
546
D:\datasets\image_agumentation\Broken\Scan 10\14-02-2022_12_05_23\scanned_image.png
555
D:\datasets\image_agumentation\Broken\Scan 11\14-02-2022_12_10_57\scanned_image.png
579
D:\datasets\image_agumentation\Broken\Scan 12\14-02-2022_12_15_57\scanned_image.png
561
D:\datasets\image_agumentation\Broken\Scan 13\14-02-2022_12_23_35\scanned_image.png
555
D:\datasets\image_agumentation\Broken\Scan 2\14-02-2022_11_09_42\scanned_image.png
554
D:\datasets\image_agumentation\Broken\Scan 3\14-02-2022_11_15_41\scanned_image.png
566
D:\datasets\image_agumentation\Broken\Scan 4\14-02-2022_11_22_59\scanned_image.png
609
D:\datasets\image_agumentation\Broken\Scan 5\14-02-2022_11_30_04\scanned_image.png
549
D:\datasets\image_agumentation\Broken\Scan 6\14-02-2022_11_36_08\scanned_image.png
567
D:\datasets\image_agumentation\Broken\Scan 7\14-02-2022_11_43_34\scanned_image.png
585
D:\datasets\image_agumentation\Broken\S

UnboundLocalError: local variable 'category_id' referenced before assignment